In [71]:
with open("inputs/Day_13.txt") as f:
    raw_input_data = f.read()

In [72]:
from collections import namedtuple
from enum import Enum


class Tile(Enum):
    Empty = 0
    Wall = 1
    Block = 2
    Paddle = 3
    Ball = 4

    
def part_1_solution(raw_input):
    puzzle_input = list(map(int, raw_input.split(',')))
    
    blocks_cnt = run_simulation(puzzle_input)
    
    return blocks_cnt


def run_simulation(program):
    blocks_cnt = 0
    computer = start_computer(program)
    item = next(computer)
    
    while item is not Finished:
        if isinstance(item, Output):
            tile = Tile(item.tile_id)
                
            if tile == Tile.Block:
                blocks_cnt += 1
                    
            item = next(computer)
        else:
            raise Exception(f"Unsuported item from generator: {item}")
            
    return blocks_cnt

    
Finished = object()
Input = object()
Output = namedtuple('Output', ('x', 'y', 'tile_id'))


def start_computer(sequence):
    index = 0
    relative_base = 0
    memory = defaultdict(int)
    output_parameters = list()
    
    for i, value in enumerate(sequence):
        memory[i] = value
        
    diag_nbr = None
    while True:
        opt_code = memory[index]
        opt_code_with_modes = str(opt_code).zfill(5)
        opt_code = int(opt_code_with_modes[-2:])
        modes = opt_code_with_modes[:-2]
        
        
        par_1_address = get_parameter_address(memory, modes[-1], index + 1, relative_base)
        par_2_address = get_parameter_address(memory, modes[-2], index + 2, relative_base)
        par_3_address = get_parameter_address(memory, modes[-3], index + 3, relative_base)
        
        par_1 = memory[par_1_address]
        par_2 = memory[par_2_address]
        
        if opt_code == 99:
            break
        elif opt_code == 3:
            target_address = par_1_address
            memory[target_address] = yield Input
            index += 2
        elif opt_code == 4:
            diag_nbr = par_1
            output_parameters.append(diag_nbr)
            
            if len(output_parameters) >= 3:
                yield Output(*output_parameters)
                output_parameters = list()
                
            index += 2
        elif opt_code == 1:
            target_address = par_3_address
            memory[target_address] = par_1 + par_2
            index += 4
        elif opt_code == 2:
            target_address = par_3_address
            memory[target_address] = par_1 * par_2
            index += 4
        elif opt_code == 5:
            if par_1 != 0:
                index = par_2
            else:
                index += 3
        elif opt_code == 6:
            if par_1 == 0:
                index = par_2
            else:
                index += 3
        elif opt_code == 7:
            target_address = par_3_address
            memory[target_address] = 1 if par_1 < par_2 else 0
            index += 4
        elif opt_code == 8:
            target_address = par_3_address
            memory[target_address] = 1 if par_1 == par_2 else 0
            index += 4
        elif opt_code == 9:
            relative_base += par_1
            index += 2
        else:
            print(f"Wrong code: {opt_code}")  
    
    yield Finished

            
def get_parameter_address(memory, mode, par_index, relative_base):
    if mode == "0":
        return memory[par_index]
    elif mode == "1":
        return par_index
    elif mode == "2":
        return relative_base + memory[par_index]
    else:
        print(f"[ERROR] Wrong mode code: {mode}")

In [73]:
print(f"Part 1 solution: {part_1_solution(raw_input_data)}")

Part 1 solution: 341


In [74]:
from collections import namedtuple
from enum import Enum


class Tile(Enum):
    Empty = 0
    Wall = 1
    Block = 2
    Paddle = 3
    Ball = 4

    
def part_2_solution(raw_input):
    puzzle_input = list(map(int, raw_input.split(',')))
    puzzle_input[0] = 2
    
    score = run_simulation(puzzle_input)
    
    return score


def run_simulation(program):
    score = 0
    computer = start_computer(program)
    item = next(computer)
    paddle_position, ball_position = None, None
    
    while item is not Finished:
        if item is Input:
            move = get_next_move(paddle_position, ball_position)
            paddle_position = (paddle_position[0] + move.value, paddle_position[1])
            item = computer.send(move.value)
        elif isinstance(item, Output):
            if item.x == -1 and item.y == 0:
                score = item.tile_id
            else:
                tile = Tile(item.tile_id)
                
                if tile == Tile.Ball:
                    ball_position = (item.x, item.y)
            
                if tile == Tile.Paddle:
                    paddle_position = (item.x, item.y)
                    
            item = next(computer)
        else:
            raise Exception(f"Unsuported item from generator: {item}")
            
    return score


def get_next_move(paddle_position, ball_position):
    diff_x = paddle_position[0] - ball_position[0]
    
    if diff_x < 0:
        return Move.Right
    elif diff_x > 0:
        return Move.Left
    
    return Move.No_Move


class Move(Enum):
    Left = -1
    No_Move = 0
    Right = 1

    
Finished = object()
Input = object()
Output = namedtuple('Output', ('x', 'y', 'tile_id'))


def start_computer(sequence):
    index = 0
    relative_base = 0
    memory = defaultdict(int)
    output_parameters = list()
    
    for i, value in enumerate(sequence):
        memory[i] = value
        
    diag_nbr = None
    while True:
        opt_code = memory[index]
        opt_code_with_modes = str(opt_code).zfill(5)
        opt_code = int(opt_code_with_modes[-2:])
        modes = opt_code_with_modes[:-2]
        
        
        par_1_address = get_parameter_address(memory, modes[-1], index + 1, relative_base)
        par_2_address = get_parameter_address(memory, modes[-2], index + 2, relative_base)
        par_3_address = get_parameter_address(memory, modes[-3], index + 3, relative_base)
        
        par_1 = memory[par_1_address]
        par_2 = memory[par_2_address]
        
        if opt_code == 99:
            break
        elif opt_code == 3:
            target_address = par_1_address
            memory[target_address] = yield Input
            index += 2
        elif opt_code == 4:
            diag_nbr = par_1
            output_parameters.append(diag_nbr)
            
            if len(output_parameters) >= 3:
                yield Output(*output_parameters)
                output_parameters = list()
                
            index += 2
        elif opt_code == 1:
            target_address = par_3_address
            memory[target_address] = par_1 + par_2
            index += 4
        elif opt_code == 2:
            target_address = par_3_address
            memory[target_address] = par_1 * par_2
            index += 4
        elif opt_code == 5:
            if par_1 != 0:
                index = par_2
            else:
                index += 3
        elif opt_code == 6:
            if par_1 == 0:
                index = par_2
            else:
                index += 3
        elif opt_code == 7:
            target_address = par_3_address
            memory[target_address] = 1 if par_1 < par_2 else 0
            index += 4
        elif opt_code == 8:
            target_address = par_3_address
            memory[target_address] = 1 if par_1 == par_2 else 0
            index += 4
        elif opt_code == 9:
            relative_base += par_1
            index += 2
        else:
            print(f"Wrong code: {opt_code}")  
    
    yield Finished

            
def get_parameter_address(memory, mode, par_index, relative_base):
    if mode == "0":
        return memory[par_index]
    elif mode == "1":
        return par_index
    elif mode == "2":
        return relative_base + memory[par_index]
    else:
        print(f"[ERROR] Wrong mode code: {mode}")

In [75]:
print(f"Part 2 solution: {part_2_solution(raw_input_data)}")

Part 2 solution: 17138
